Initializations

In [ ]:
import os
import math
import collections
from   collections import defaultdict
import sys
import string
import sys
import time
import nltk
from nltk import PorterStemmer
from nltk.corpus import stopwords
from collections import Counter
#nltk.download()
stemmer = PorterStemmer()
token = nltk.tokenize.RegexpTokenizer(r'\w+')

dir = 'C:/Users/Yoav/Desktop/reuters/training/'
queriesDir = 'C:/Users/Yoav/Desktop/HW2IRFinal/HW2IRFinal/input/queries.txt'
resultDir = 'C:/Users/Yoav/Desktop/HW2IRFinal/HW2IRFinal/result.txt'

Pre-Process - analyzeLine and buildLists functions :


analyzeLine - This function get a line in input and return a ready line for index. 
Each line passes through the following stages:
- tokenization
- stemming
- stop- words removal 

buildLists - Finally we save all words in 2 lists for the next steps:
- words_list - unique values of all words in all documents together.
- all_words_list - all words that appear in all documents. 

In [ ]:
def analyzeLine (line):
    finalLine = [] 
    ans=[]
    row = nltk.word_tokenize(line.lower()) 
    
    #pre-process for each word
    for word in row :
        
        #stemming
        stemWord = stemmer.stem(word) 
        
        #tokenize
        if any(w.isdigit() for w in stemWord): 
            tokenWord = [stemWord] 
        else: 
            tokenWord = token.tokenize(stemWord)
        
        #stop- words removal 
        for wword in tokenWord:
            if wword not in nltk.corpus.stopwords.words('english'):
                finalLine.append(wword) #bulid final line
    
    return (finalLine)

#build lists of words: "words_list" only unique values, and "all_words_list" include all words in documents  
def buildLists (line):
    for word in line: 
        if word not in words_list[fname]:
            words_list[fname].append(word)
        all_words_list[fname].append(word)
    
    return

tfIDFarrays and Inverted Index Functions:

- The first function build all connected arrays that important for build the inverted index.
Inverted index is calculated by tf*idf weights model:
TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).
"tf" array save for each word the number of times it appear in each document. Both TF's parts are taken from "tf" array. 
IDF(t) = log_e(Total number of documents / Number of documents with term t in it).
Total number of documents - "NumOfDocs" variable.
second part of idf is taken from "DocPerTerm" array, that keep for each word the number of documents it appears in it. 
The function also calculated for each doc it's vector length.

- The second function build the appropriate inverted index, according to the documents in the input file. 
The calculate is for each word and document :tf*idf.
the index is look like this : "temporao: [0.0, 0.0]". meaning this word is appear in 2 documents the same number of times this is why the weight is 0. 

In [ ]:
def tfIDFarrays ():
        
    for key in words_list.keys():
        for word in words_list[key]:
            # tf array
            tf[key].append(all_words_list[key].count(word))
            
            # calculated DocPerTerm array 
            if not word in DocPerTerm:
                DocPerTerm[word] = 1
            else:
                DocPerTerm[word] += 1
                
    # calculate IDF array 
    for word in DocPerTerm :
        idf[word] = math.log(float(NumOfDocs)/float(DocPerTerm[word]),10) 
        
    for doc in words_list:
        totalWords = len(all_words_list[doc])
        fi=0
        for word in words_list[doc]:
            fi += math.pow(all_words_list[doc].count(word),2)/totalWords
        VecPerDoc[doc] = fi
        
    return 


def invertedIndex():
    
    for key in words_list.keys():
        word_enum = 0
        for word in words_list[key]:
            weightForIndex = {}
            weightForIndex[key] = float(tf[key][word_enum])/float(max(tf[key]))
            inverted_index[word].append(weightForIndex)
            word_enum = word_enum + 1

    return

Main loop - Part I:
Read documents, analyze document and build the inverted index

In [ ]:
words_list = defaultdict(list)
all_words_list = defaultdict(list)
tf = defaultdict(list)
inverted_index = defaultdict(list)
DocPerTerm = defaultdict(list)
VecPerDoc = defaultdict(list)
idf = defaultdict(list)
NumOfDocs = 0 

for root, dirs, files in os.walk(dir):
    for fname in files:
        current_file = "%s%s%s" % (os.path.abspath(root), os.path.sep, fname)
        f = open (current_file, 'r')
        
        for line in f:
            row = [] 
            row = analyzeLine (line) #tokenize, stemming and stop-words for line and build words lists
            buildLists (row)
        
    f.close ()
    NumOfDocs = len(words_list)
    tfIDFarrays () #calculate tf idf array 
    invertedIndex() #build inverted index
  

Query Function:

getDocResult function return a list of documantes with the similarity between query and documents. Output documents are ranked acording to aimilarity to query.

First we analyze the query. Each query passes through the following stages:

    tokenization
    stemming
    stop- words removal

Finally we save all words in 2 lists for the next steps:

    query_list - unique values of all words in query.
    all_query_list - all words that appear in the query.

Calculated Similarity:
Cosine Similarity measures the consine of the angle between two vectors. First we calculated for the query the inner product = sum of tf(doc)*ifd(word in query). Then for each doc we calculated CosSim=inner priduct normilized by vector lenghts


In [ ]:
def getDocResult(query):
    query_words = defaultdict(list)
    all_query_words = defaultdict(list)
    res= defaultdict(list)
    result= defaultdict(list)
    finalQuery = []
    totalQueryWords = 0
    
    # analyze Query
    finalQuery = analyzeLine (query)  
    
    #build lists of words: "query_words" only unique values, and "all_query_words" include all words in documents           
    for word in finalQuery: 
        
        if word not in query_words[word]:
            query_words[word].append(word)
        all_query_words[word].append(word)
    
    #Count number of words in query
    for word in all_query_words:
        totalQueryWords =totalQueryWords + all_query_words[word].count(word)

    # Calculated Similarity 
    for word in query_words:
        for tf in inverted_index[word]:
            for doc in tf:
                if not doc in res:
                    res[doc] = (float(tf[doc]) * float(idf[word]))
                else:
                    res[doc] += (float(tf[doc]) * float(idf[word]))

        
        for doc in res:
            result[doc] = res[doc] / math.sqrt(totalQueryWords*VecPerDoc[doc]) 
            
   
    return result
        

Function runQuery
The function is getting a query and return the top 20 relevant documents ranked according to the Similarity measure.

Function getTop20
The function read a query file and write to a result file for each query the top 20 relevant documents ranked according to the Similarity measure.

In [ ]:
def runQuery(query):
    doc_result = defaultdict(list)
    sort_doc_result = defaultdict(list)
    
    doc_result = getDocResult(query)
    sort_doc_result = sorted(doc_result,key=doc_result.get,reverse=True)[:20]
    
    return (";").join(sort_doc_result)

def getTop20():
    results_file = open(resultDir, "a")
    queries =  open(queriesDir,'r')
    for query in queries: #run on each query
        if query != '\n':
            results_file.write(runQuery(query) + '\n')
    return 



Main loop - Part II:
In order to create result output, pease run getTop20()

In [ ]:
getTop20()